In [7]:
#!pip install requests
#!pip3 install Beautifulsoup4

In [20]:
import requests
from bs4 import BeautifulSoup
from math import *
import re
import numpy as np
import time

In [2]:
URL = 'https://www.airlinequality.com/review-pages/a-z-airline-reviews/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
tabs_content = soup.find('div', class_="tabs-content")
airlines_tag = tabs_content.find_all('li')

In [6]:
#collect the different urls for each airline company
airlines_ref=[]
for element in airlines_tag :
    tag = element.find('a')
    airlines_ref.append(tag['href'])
    

In [10]:
test = ['/airline-reviews/ab-aviation','/airline-reviews/air-france']

In [34]:
def get_url_airline(airlines):
    
    """
    Returns the sum of two decimal numbers in binary digits.

    Parameters
    ----------
    airlines (list): A list of airlines reference needed, ex : '/airline-reviews/ab-aviation'

    Returns
    -------
    A list of all url reviews linked to the companies needed   
    """
    
    URL = 'https://www.airlinequality.com'
    
    total_url=[]
    
    for airline in airlines :
        
        url_company ={'airline':[], 'seats':[]} #
        url = URL + airline
        page = requests.get(url)
        time.sleep(1)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        #Test if onglet seat and airline
        seat_content = soup.find('li', class_=["tab-title seat","tab-title seat active"])
        airline_content = soup.find('li', class_=["tab-title airline","tab-title airline active"])
        
        if not(airline_content == None):
            ref_airline = airline_content.find('a')['href']
            new_url = URL + ref_airline + '/?sortby=post_date%3ADesc&pagesize=100'
            new_page = requests.get(new_url)
            #time.sleep(1)
            new_soup = BeautifulSoup(new_page.content, 'html.parser')
            
            test_page=new_soup.find(
                    'article',class_="comp comp_reviews-pagination querylist-pagination position-")
            
            airline_url = []
            if not(test_page == None):
                query_page = new_soup.find('div', class_='pagination-total').get_text()
                
                if not(query_page ==None):
                    number_review = int(query_page.split(" ")[-2])
                    number_page = ceil(number_review/100)

                    for i in range(number_page) :
                        url = URL + ref_airline + '/page/'+ str(i+1) + '/?sortby=post_date%3ADesc&pagesize=100'
                        if url not in airline_url:
                            airline_url.append(url)
                        
            else : airline_url.append(new_url) #only one page
        
            url_company['airline']=airline_url

        if not(seat_content == None):
            ref_seat = seat_content.find('a')['href']
            new_url = URL + ref_seat + '/?sortby=post_date%3ADesc&pagesize=100'
            new_page = requests.get(new_url)
            #time.sleep(1)
            new_soup = BeautifulSoup(new_page.content, 'html.parser')
            
            test_page=new_soup.find(
                    'article',class_="comp comp_reviews-pagination querylist-pagination position-")
            
            seat_url = []
            if not(test_page == None):
                query_page = new_soup.find('div', class_='pagination-total').get_text()
                
                if not(query_page ==None):
                    number_review = int(query_page.split(" ")[-2])
                    number_page = ceil(number_review/100)

                    for i in range(number_page) :
                        url = URL + ref_seat + '/page/'+ str(i+1) + '/?sortby=post_date%3ADesc&pagesize=100'
                        if url not in seat_url:
                            seat_url.append(url)
                        
            else : seat_url.append(new_url) # only one page
                
            url_company['seats']=seat_url
            
        total_url.append(url_company)
    return total_url

get_url_airline(test)

[{'airline': ['https://www.airlinequality.com/airline-reviews/ab-aviation/?sortby=post_date%3ADesc&pagesize=100'],
  'seats': []},
 {'airline': ['https://www.airlinequality.com/airline-reviews/air-france/page/1/?sortby=post_date%3ADesc&pagesize=100',
   'https://www.airlinequality.com/airline-reviews/air-france/page/2/?sortby=post_date%3ADesc&pagesize=100',
   'https://www.airlinequality.com/airline-reviews/air-france/page/3/?sortby=post_date%3ADesc&pagesize=100',
   'https://www.airlinequality.com/airline-reviews/air-france/page/4/?sortby=post_date%3ADesc&pagesize=100',
   'https://www.airlinequality.com/airline-reviews/air-france/page/5/?sortby=post_date%3ADesc&pagesize=100',
   'https://www.airlinequality.com/airline-reviews/air-france/page/6/?sortby=post_date%3ADesc&pagesize=100',
   'https://www.airlinequality.com/airline-reviews/air-france/page/7/?sortby=post_date%3ADesc&pagesize=100',
   'https://www.airlinequality.com/airline-reviews/air-france/page/8/?sortby=post_date%3ADesc&p

In [59]:
def generate_page_reviews(URL):
    # Get page content
    page = requests.get(URL)
    # Wait for 5 seconds
    #time.sleep(1)
    soup = BeautifulSoup(page.content, 'html.parser')
    # Get the list of reviews in the page
    reviews = soup.find_all('article', itemprop='review')
    
    dates_published = [] # Get list of publication dates
    global_ratings = [] # Get list of global ratings
    reviews_titles = [] # Get reviews' titles
    customers_countries = [] # Get country of customers
    reviews_body = [] # Get reviews' bodies
    is_verified = [] # Get boolean variable verified
    aircraft = []
    type_traveller = []
    seat_type = []
    route = []
    date_flown = []
    is_recommended = []
    for i, review in enumerate(reviews):
        dates_published.append(review.find('meta', itemprop='datePublished')['content'])
        
        global_ratings.append(review.find('span', itemprop='ratingValue').text)
        
        reviews_titles.append(review.find('h2', class_='text_header').text.strip())
                              
        temp = review.find('h3', class_='text_sub_header userStatusWrapper').text
        country = re.search('\((.*)\)', temp)
        if country is not None:
            customers_countries.append(country.group(1))
        
        if review.find('div', class_='text_content').find('em') is not None :# == 'Trip Verified':
            is_verified.append(True)
            review_body = review.find('div', class_='text_content').text.split("|",1)[1][2:] 
        else:
            is_verified.append(False)
            review_body = review.find('div', class_='text_content').text
            
        reviews_body.append(review_body)
            
        review_ratings = review.find_all('tr')
        for rating in review_ratings:
            # Aircraft
            if rating.find('td', class_='review-rating-header aircraft') is not None:
                aircraft.append(rating.find('td', class_='review-value').text)
            # Traveller type
            if rating.find('td', class_='review-rating-header type_of_traveller') is not None:
                type_traveller.append(rating.find('td', class_='review-value').text)
            # Seat type
            if rating.find('td', class_='review-rating-header cabin_flown') is not None:
                seat_type.append(rating.find('td', class_='review-value').text)
            # Route
            if rating.find('td', class_='review-rating-header route') is not None:
                route.append(rating.find('td', class_='review-value').text)
            # Date flown
            if rating.find('td', class_='review-rating-header date_flown') is not None\
            and len(rating.find('td', class_='review-value').text) == 10:
                date_flown.append(rating.find('td', class_='review-value').text)
            '''# Seat comfort
            if rating.find('td', class_='review-rating-header seat_comfort') is not None:
                seat_comfort.append(rating.find('td', class_='review-value').text)
            # Cabin staff service
            if rating.find('td', class_='review-rating-header cabin_staff_service') is not None:
                cabin_staff_service.append(rating.find('td', class_='review-value').text)
            # Food and beverages
            if rating.find('td', class_='review-rating-header food_and_beverages') is not None:
                food_beverages.append(rating.find('td', class_='review-value').text)'''
            # Is recommended
            if rating.find('td', class_='review-rating-header recommended') is not None:
                temp = rating.find_all('td')[1].text
                if temp == "yes": 
                    is_recommended.append(True)
                else: 
                    is_recommended.append(False)

        if len(dates_published) != i+1: dates_published.append(None)
        if len(global_ratings) != i+1: global_ratings.append(None)
        if len(reviews_titles) != i+1: reviews_titles.append(None)
        if len(customers_countries) != i+1: customers_countries.append(None)
        if len(reviews_body) != i+1: reviews_body.append("")
        if len(is_verified) != i+1: is_verified.append(None)
        if len(aircraft) != i+1: aircraft.append(None)
        if len(type_traveller) != i+1: type_traveller.append(None)
        if len(seat_type) != i+1: seat_type.append(None)
        if len(route) != i+1: route.append(None)
        if len(date_flown) != i+1: date_flown.append(None)
        if len(is_recommended) != i+1: is_recommended.append(None)
            
    return np.array([dates_published,
                    global_ratings,
                    reviews_titles,
                    customers_countries,
                    reviews_body,
                    is_verified,
                    aircraft,
                    type_traveller,
                    seat_type,
                    route,
                    date_flown,
                    is_recommended]).transpose()
    

In [8]:
generate_page_reviews('https://www.airlinequality.com/airline-reviews/ab-aviation/?sortby=post_date%3ADesc&pagesize=100')

array([['2019-11-11', '9', 'pretty decent airline', 'Netherlands',
        "Moroni to Moheli. Turned out to be a pretty decent airline. Online booking worked well, checkin and boarding was fine and the plane looked well maintained. Its a very short flight - just 20 minutes or so so i didn't expect much but they still managed to hand our a bottle of water and some biscuits which i though was very nice. Both flights on time.",
        True, None, 'Solo Leisure', 'Economy Class', 'Moroni to Moheli',
        'November 2019', True],
       ['2019-06-25', '1', 'Not a good airline', 'United Kingdom',
        'oroni to Anjouan. It is a very small airline. My ticket advised me to turn up at 0800hrs which I did. There was confusion at this small airport. I was then directed to the office of AB Aviation which was still closed. It opened at 0900hrs and I was told that the flight had been put back to 1300hrs and that they had tried to contact me. This could not be true as they did not have my phone

In [18]:
a = np.array([[1,2],[3,4]])
b = np.array([[5,6]]).transpose()
c=np.array(size=(1,2))
np.concatenate((None,a,b),axis=1)

TypeError: array() missing required argument 'object' (pos 1)

In [68]:
def generate_dataset(airlines):
    airlines_urls = get_url_airline(airlines)
    
    airline_data = None
    for i, airline_url in enumerate(airlines_urls):
        for url_in_airline in airline_url['airline']:
            page_data = generate_page_reviews(url_in_airline)
            print(page_data.shape)
            is_airline_review = np.ones((len(page_data),1), dtype=np.int8) # Column is_airline_comment
            airline_name = np.repeat([airlines[i].split('/')[-1]], len(page_data)).reshape((len(page_data),1))
            page_data = np.concatenate((page_data, is_airline_review, airline_name), axis=1)
            if airline_data is None:
                airline_data = page_data
            else:
                airline_data = np.concatenate((airline_data, page_data))
                
        for url_in_airline in airline_url['seats']:
            page_data = generate_page_reviews(url_in_airline)
            is_airline_review = np.zeros((len(page_data),1), dtype=np.int8) # Column is_airline_comment
            airline_name = np.repeat([airlines[i].split('/')[-1]], len(page_data)).reshape((len(page_data),1))
            page_data = np.concatenate((page_data, is_airline_review, airline_name), axis=1)
            if airline_data is None:
                airline_data = page_data
            else:
                airline_data = np.concatenate((airline_data, page_data))
                
    return airline_data
            
print(generate_dataset([test[1]]))  

(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(59, 12)
[['2020-10-19' '1' '"refused to let my mother board a flight"' ... False
  1 'air-france']
 ['2020-10-15' '10' '"I was very impressed"' ... True 1 'air-france']
 ['2020-10-15' '2' '"not allowed on board"' ... False 1 'air-france']
 ...
 ['2008-12-10' '5' '"very tight seat pitch"' ... False 0 'air-france']
 ['2008-11-17' '1' '"seats felt very cramped"' ... False 0 'air-france']
 ['2008-09-29' '1' '"would not recommend these seats"' ... False 0
  'air-france']]


In [67]:
test

['/airline-reviews/ab-aviation', '/airline-reviews/air-france']

In [69]:
data = generate_dataset(test)

(3, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(100, 12)
(59, 12)


In [84]:
data[:,4]

array(["Moroni to Moheli. Turned out to be a pretty decent airline. Online booking worked well, checkin and boarding was fine and the plane looked well maintained. Its a very short flight - just 20 minutes or so so i didn't expect much but they still managed to hand our a bottle of water and some biscuits which i though was very nice. Both flights on time.",
       'oroni to Anjouan. It is a very small airline. My ticket advised me to turn up at 0800hrs which I did. There was confusion at this small airport. I was then directed to the office of AB Aviation which was still closed. It opened at 0900hrs and I was told that the flight had been put back to 1300hrs and that they had tried to contact me. This could not be true as they did not have my phone number. I was with a local guide and he had not been informed either. I presume that I was bumped off. The later flight did operate but as usual, there was confusion at check-in. The flight was only 30mins and there were no further problems

In [78]:
np.savetxt('./airlines_data.csv', data, fmt='%s', encoding='utf-8', header = 'date_published@; \
                                                  global_ratings@;\
                                                  reviews_titles@;\
                                                  customers_countries@;\
                                                  reviews_body@;\
                                                  is_verified@;\
                                                  aircraft@;\
                                                  type_traveller@;\
                                                  seat_type@;\
                                                  route@;\
                                                  date_flown@;\
                                                  is_recommended@;\
                                                  is_airline_review@;\
                                                  airline_name', delimiter = "@;")

In [22]:

URL1 = 'https://www.airlinequality.com'
dict_reviews={}
for airline in airlines_ref:
    new_url = URL1 + airline
    new_page = requests.get(new_url)
    new_soup = BeautifulSoup(new_page.content, 'html.parser')
    
    seat_content = new_soup.find('li', class_=["tabs-title seat","tabs-title seat active"])
    print(seat_content)
    print(airline.split('/')[-1])
    if not(seat_content == None):
     #   dict_reviews[airline.split('/')[-1]]=[]
        
      #  col_content = soup.find('div', class_='col-content'
        
        
    break
    
    

None
ab-aviation


In [141]:
airline='/airline-reviews/air-france'

url2 = URL1 + airline
new_page = requests.get(url2)
new_soup = BeautifulSoup(new_page.content, 'html.parser')


In [158]:
seat_content = new_soup.find('li', class_="tab-title seat")
if not(seat_content == None):
    dict_reviews[airline.split('/')[-1]]=[]
    
    url3 = URL1 + seat_content.find('a')['href'] +
    page2 = requests.get(url3)
    soup2 = BeautifulSoup(page2.content, 'html.parser')
    
    #col_content = soup2.find('div', id='container')
    col_content = soup2.find('section', class_="layout-section layout-2 closer-top")
    col_content = col_content.find('div', class_='col-content')

    #col_content = col_content.find('div', class_='layout-section layout-2 closer-top')

    #col_content = col_content.find('article', class_="comp comp_reviews-airline querylist position-content ")
    
    review = col_content.find_all('div', class_="text_content")
    
    for comment in review :
        dict_reviews[airline.split('/')[-1]].append(comment.get_text())
        
print(dict_reviews)


{'air-france': ["✅ Trip Verified |  The business class seat had plenty of room, and I liked that it was a pod-style (so I had decent privacy). My biggest complaint is the comfort when in bed mode it is very lumpy with the lumbar protruding into your back.  AF has invested a lot into these seats, and they could be great if they would only take the time to work on them a bit. A lay-flat bed that protrudes into your back is not OK. Tried using an extra blanket to make the bed more comfy but it didn't work. Please Air France, improve the comfort of the cushions for bed mode.", '✅ Trip Verified |  I had done online check in for economy seats fairly forward but at boarding gate in paris they changed the seat to somewhere at the back where the planes width narrows. The seat was so small & rigid & once the guy in front of me reclined his seat i felt so claustrophobic. For the sake on entertainment in an 11hr plus flight which was further delayed by 2hrs was a tiny screen with little content & 

In [152]:
len(review)

10

In [154]:
dict_reviews

{'air-france': [None, None, None, None, None, None, None, None, None, None]}

In [ ]:
if not(seat_content == None):
    dict_reviews[airline.split('/')[-1]]=[]

    col_content = soup.find('div', class_='col-content')

print(col_content)